In [ ]:
# Dependencies and Setup
import pandas as pd
import requests
# import numpy as np
import io
# import ipyleaflet
import json
from pathlib import Path
from pprint import pprint
# from ipyleaflet import Map, Marker, LayersControl, SearchControl, FullScreenControl, WidgetControl, GeoJSON, LayerGroup
# from geopy.exc import GeocoderTimedOut 
# from geopy.geocoders import Nominatim 
# from ipywidgets import link, FloatSlider
# from branca.colormap import linear
from pandas_geojson import to_geojson
# from ipywidgets import HTML, widgets

In [ ]:
# Downloading the csv file from the GitHub repository
url = "https://raw.githubusercontent.com/brown016340/Project---3/benjamin/data/coords.csv"
download = requests.get(url).content

# Reading the downloaded content and turning it into a pandas dataframe
coords_df = pd.read_csv(io.StringIO(download.decode('utf-8')))

# Printing out the first 5 rows of the dataframe
coords_df.head()

In [ ]:
# Downloading the csv file from the GitHub repository
url = "https://raw.githubusercontent.com/brown016340/Project---3/benjamin/data/2019cleaned.csv"
download = requests.get(url).content

# Reading the downloaded content and turning it into a pandas dataframe
happy_df = pd.read_csv(io.StringIO(download.decode('utf-8')))

# Printing out the first 5 rows of the dataframe
happy_df.head()

In [ ]:
print("Columns in happy15_df:", happy_df.columns)
print("Columns in coords_df:", coords_df.columns)

In [ ]:
# Join the dataframes
data_df = pd.merge(happy_df, coords_df, on='Country', how='inner')
data_df.head()

In [ ]:
# Convert to geo json
geo_json = to_geojson(df=data_df, lat='Latitude', lon='Longitude',
                      properties=['Country','Happiness_Rank','Happiness_Score', 'Economy_GDP_per_Capita', 'Family', 'Health_Life_Expectancy', 'Freedom', 'Trust_Government_Corruption', 'Generosity'])
pprint(geo_json)

In [ ]:
# Function to add 'id' field to each feature
def add_id_to_features(json_data):
    for feature in json_data['features']:
        # Assuming 'Country' is unique for each feature
        country_name = feature['properties'].get('Country', 'Unknown')
        # Create a simple identifier based on the first three characters of the country name
        feature['id'] = country_name[:2].upper()

# Call the function to add 'id' field
add_id_to_features(geo_json)

In [ ]:
# Load the country geojson that has polygon data for each country
def load_data(url, filename, file_type):
    r = requests.get(url)
    with open(filename, 'w') as f:
        f.write(r.content.decode("utf-8"))
    with open(filename, 'r') as f:
        return file_type(f)

geo_json_data = load_data(
    'https://raw.githubusercontent.com/brown016340/Project---3/benjamin/data/countries.geojson',
    'countries.json',
     json.load)

In [ ]:
# Add the polygon coords to the original json
admin_country_mapping = {feature['properties']['ADMIN']: feature for feature in geo_json_data['features']}

# Update the geometry in geo_json based on the mapping
for feature in geo_json['features']:
    country_name = feature['properties']['Country']
    if country_name in admin_country_mapping:
        new_geometry = admin_country_mapping[country_name]['geometry']
        feature['geometry'] = new_geometry

In [ ]:
# Serializing json
json_object = json.dumps(geo_json)
 
# Writing to sample.json
with open("C:/Users/browba/Documents/2019.geo.json", "w") as outfile:
    outfile.write(json_object)